In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics, load_weights_and_evaluate
from model_builders import GCN_pretraining
from hyperparameter_tuning_GCN import objective
from functools import partial
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import dill
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\tomas\miniconda3\envs\binding\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\tomas\miniconda3\envs\binding

In [2]:
# Specify model callbacks on training
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)

model_params = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [32,64,96],
        "fp_length" : [96,96,96],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [96,96,48],
        'dropout_rate' : [0.1,0.1],
        'lr' : 0.001,
        'batch_size' : int(64),
        'n_epochs' : int(5)
        }


In [3]:
fspace = {
    'conv1' : hp.quniform('conv1', 32, 96, 8),
    'conv2' : hp.quniform('conv2', 48, 128, 8),
    'conv3' : hp.quniform('conv3', 64, 168, 8),
    'fp' : hp.quniform('fp', 64, 196, 8),
    'dense1' : hp.quniform('dense1',96,256,32),
    'dense2' : hp.quniform('dense2',96,256,32),
    'dense3' : hp.quniform('dense3',48,128,32),
    'dropout_rate' : hp.uniform('dropout_rate',0.1,0.5),
    'lr' : hp.uniform('lr',0.0005,0.01),
    'n_epochs' : hp.quniform('n_epochs',15,40,5) 
}

In [4]:
target_1 = 'pi3k'
base_path_1 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_1 = base_path_1+f'/data/{target_1}/data.csv'
df_p38=pd.read_csv(data_fpath_1).set_index('biolab_index')

with open(base_path_1+f'/data/{target_1}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_p38 = dill.load(in_f)

with open(base_path_1+f'/data/{target_1}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_p38 = dill.load(in_f)
    
target_2 = 'akt1'
base_path_2 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_2 = base_path_2+f'/data/{target_2}/data.csv'
df_akt1 = pd.read_csv(data_fpath_2).set_index('biolab_index')

with open(base_path_2+f'/data/{target_2}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_akt1 = dill.load(in_f)
with open(base_path_2+f'/data/{target_2}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_akt1 = dill.load(in_f)
    
target_3 = 'pi3k'
base_path_3 = f'C:/Users/tomas/Documents/GitHub/kinase_binding'

data_fpath_3 = base_path_3+f'/data/{target_3}/data.csv'
df_pi3k = pd.read_csv(data_fpath_3).set_index('biolab_index')

with open(base_path_3+f'/data/{target_3}/train_val_folds.pkl', "rb") as in_f:
    train_val_folds_pi3k = dill.load(in_f)
with open(base_path_3+f'/data/{target_3}/train_test_folds.pkl', "rb") as in_f:
    train_test_folds_pi3k = dill.load(in_f)
    

In [5]:
#Evaluation Splits (our test set)
training_p38 = df_p38.loc[train_test_folds_p38[0]]
validation_p38 = df_p38.loc[train_test_folds_p38[1]]


training_akt1 = df_akt1.loc[train_test_folds_akt1[0]]
validation_akt1 = df_akt1.loc[train_test_folds_akt1[1]]
               

training_pi3k = df_pi3k.loc[train_test_folds_pi3k[0]]
validation_pi3k = df_pi3k.loc[train_test_folds_pi3k[1]]

In [6]:
#AVE Bias splits (test) only p38
ave_p38_train = pd.read_csv('data/p38/split_aveb/train_all.csv', index_col=0)
ave_p38_val = pd.read_csv('data/p38/split_aveb/test.csv', index_col = 0)
print(len(ave_p38_train),len(ave_p38_val))

3186 371


In [7]:
#Random splits with sklearn (on our test set)
df_p38 = df_p38.reset_index(drop=True)
X_train_p38, X_val_p38, Y_train_p38, Y_val_p38 = train_test_split(df_p38.rdkit,
                                                                  df_p38.Binary,
                                                                  test_size = 0.15,
                                                                  train_size = 0.85,
                                                                  shuffle = True)
X_train_p38 = pd.DataFrame(X_train_p38)
X_val_p38 = pd.DataFrame(X_val_p38)
print(len(X_train_p38),len(X_val_p38))

3190 564


In [8]:
df_akt1 = df_akt1.reset_index(drop=True)
X_train_akt1, X_val_akt1, Y_train_akt1, Y_val_akt1 = train_test_split(df_akt1.rdkit,
                                                                     df_akt1.Binary,
                                                                     test_size = 0.15,
                                                                     train_size = 0.85,
                                                                     shuffle = True)
X_train_akt1 = pd.DataFrame(X_train_akt1)
X_val_akt1 = pd.DataFrame(X_val_akt1)
print(len(X_train_akt1),len(X_val_akt1))

1819 321


In [9]:
df_pi3k = df_pi3k.reset_index(drop=True)
X_train_pi3k, X_val_pi3k, Y_train_pi3k, Y_val_pi3k = train_test_split(df_pi3k.rdkit,
                                                                      df_pi3k.Binary,
                                                                      test_size = 0.15,
                                                                      train_size = 0.85,
                                                                      shuffle = True)
X_train_pi3k = pd.DataFrame(X_train_pi3k)
X_val_pi3k = pd.DataFrame(X_val_pi3k)
print(len(X_train_pi3k),len(X_val_pi3k))

3190 564


In [ ]:
fmin_objective = partial(objective, train_sets = training_p38, val_sets = validation_p38)

In [ ]:
def run_trials():

    trials_step = 0  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 0  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("gcn.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("gcn.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [ ]:
trials = run_trials()

In [ ]:
best_loss = trials.trials[0]['result']['loss']
for i in range(1,len(trials.trials)):
    if (trials.trials[i]['result']['loss'] <=  best_loss):
        best_loss = trials.trials[i]['result']['loss']
        index = i
best_params = trials.trials[index]['misc']['vals']

In [ ]:
from hyperparameter_tuning_GCN import GCN_hyper
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)
gcn_best = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [int(best_params['conv1'][0]), int(best_params['conv2'][0]), int(best_params['conv3'][0])],
        "fp_length" : [int(best_params['fp'][0]), int(best_params['fp'][0]), int(best_params['fp'][0])],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [int(best_params['dense1'][0]), int(best_params['dense2'][0]), int(best_params['dense3'][0])],
        'dropout_rate' : [best_params['dropout_rate'][0], best_params['dropout_rate'][0]],
        'lr' : best_params['lr'][0],
        'batch_size' : int(64),
        'n_epochs' : int(best_params['n_epochs'][0])
        }
gcn = GCN_hyper(gcn_best)

In [ ]:
train_list_p38 = [training_p38, ave_p38_train, X_train_p38]
val_list_p38 = [validation_p38, ave_p38_val, X_val_p38]

train_list_akt1 = [training_akt1, X_train_akt1]
val_list_akt1 = [validation_akt1, X_val_akt1]

train_list_pi3k = [training_pi3k, X_train_pi3k]
val_list_pi3k = [validation_pi3k, X_val_pi3k]

In [ ]:
eval_p38 = {}
#eval_akt1 = {}
#eval_pi3k = {}
for i in range(len(train_list_p38)):
    if i == 2:
        X_atoms_cold,X_bonds_cold,X_edges_cold = gcn.dataframe_to_gcn_input(val_list_p38[i])
        Y_cold = Y_val_p38
        X_atoms_train, X_bonds_train, X_edges_train = gcn.dataframe_to_gcn_input(train_list_p38[i])
        Y = Y_train_p38
    else:
        X_atoms_cold,X_bonds_cold,X_edges_cold = gcn.dataframe_to_gcn_input(val_list_p38[i])
        Y_cold = val_list_p38[i].Binary
        X_atoms_train, X_bonds_train, X_edges_train = gcn.dataframe_to_gcn_input(train_list_p38[i])
        Y = train_list_p38[i].Binary    
    
    gcn_encoder = gcn.build_encoder()
    gcn_model = gcn.build_model(gcn_encoder)
    gcn_model.fit([X_atoms_train,X_bonds_train,X_edges_train],Y,
                  batch_size = gcn_best['batch_size'],
                  epochs = gcn_best['n_epochs'],
                  verbose = 1,
                  shuffle=True,
                  validation_data = ([X_atoms_cold,X_bonds_cold,X_edges_cold],Y_cold))
    y_pred_val = gcn_model.predict([X_atoms_cold,X_bonds_cold,X_edges_cold])
    if i == 0:
        eval_p38['Test'] = calculate_metrics(np.array(Y_cold), y_pred_val.squeeze())
    elif i == 1:
        eval_p38['Ave'] = calculate_metrics(np.array(Y_cold), y_pred_val.squeeze())
    elif i == 2:
        eval_p38['Random'] = calculate_metrics(np.array(Y_cold), y_pred_val.squeeze())
        
    


In [ ]:
eval_p38 = pd.DataFrame(eval_p38).T
eval_p38.to_csv('../../../../Desktop/binding/thesis english/Results/1-DeepGCN/p38.csv')
eval_p38

In [ ]:
eval_akt1 = {}
for i in range(len(train_list_akt1)):
    if i == 1:
        X_atoms_cold,X_bonds_cold,X_edges_cold = gcn.dataframe_to_gcn_input(val_list_akt1[i])
        Y_cold = Y_val_akt1
        X_atoms_train, X_bonds_train, X_edges_train = gcn.dataframe_to_gcn_input(train_list_akt1[i])
        Y = Y_train_akt1
    else:
        X_atoms_cold,X_bonds_cold,X_edges_cold = gcn.dataframe_to_gcn_input(val_list_akt1[i])
        Y_cold = val_list_akt1[i].Binary
        X_atoms_train, X_bonds_train, X_edges_train = gcn.dataframe_to_gcn_input(train_list_akt1[i])
        Y = train_list_akt1[i].Binary    
    
    gcn_encoder = gcn.build_encoder()
    gcn_model = gcn.build_model(gcn_encoder)
    gcn_model.fit([X_atoms_train,X_bonds_train,X_edges_train],Y,
                  batch_size = gcn_best['batch_size'],
                  epochs = gcn_best['n_epochs'],
                  verbose = 1,
                  shuffle=True,
                  validation_data = ([X_atoms_cold,X_bonds_cold,X_edges_cold],Y_cold))
    y_pred_val = gcn_model.predict([X_atoms_cold,X_bonds_cold,X_edges_cold])
    if i == 0:
        eval_akt1['Test'] = calculate_metrics(np.array(Y_cold), y_pred_val.squeeze())
    elif i == 1:
        eval_akt1['Random'] = calculate_metrics(np.array(Y_cold), y_pred_val.squeeze())

In [ ]:
eval_akt1 = pd.DataFrame(eval_akt1).T
eval_akt1.to_csv('../../../../Desktop/binding/thesis english/Results/1-DeepGCN/akt1.csv')
eval_akt1

In [ ]:
eval_pi3k = {}
for i in range(len(train_list_pi3k)):
    if i == 1:
        X_atoms_cold,X_bonds_cold,X_edges_cold = gcn.dataframe_to_gcn_input(val_list_pi3k[i])
        Y_cold = Y_val_pi3k
        X_atoms_train, X_bonds_train, X_edges_train = gcn.dataframe_to_gcn_input(train_list_pi3k[i])
        Y = Y_train_pi3k
    else:
        X_atoms_cold,X_bonds_cold,X_edges_cold = gcn.dataframe_to_gcn_input(val_list_pi3k[i])
        Y_cold = val_list_pi3k[i].Binary
        X_atoms_train, X_bonds_train, X_edges_train = gcn.dataframe_to_gcn_input(train_list_pi3k[i])
        Y = train_list_pi3k[i].Binary    
    
    gcn_encoder = gcn.build_encoder()
    gcn_model = gcn.build_model(gcn_encoder)
    gcn_model.fit([X_atoms_train,X_bonds_train,X_edges_train],Y,
                  batch_size = gcn_best['batch_size'],
                  epochs = gcn_best['n_epochs'],
                  verbose = 1,
                  shuffle=True,
                  validation_data = ([X_atoms_cold,X_bonds_cold,X_edges_cold],Y_cold))
    y_pred_val = gcn_model.predict([X_atoms_cold,X_bonds_cold,X_edges_cold])
    if i == 0:
        eval_pi3k['Test'] = calculate_metrics(np.array(Y_cold), y_pred_val.squeeze())
    elif i == 1:
        eval_pi3k['Random'] = calculate_metrics(np.array(Y_cold), y_pred_val.squeeze())

In [ ]:
eval_pi3k = pd.DataFrame(eval_pi3k).T
eval_pi3k.to_csv('../../../../Desktop/binding/thesis english/Results/1-DeepGCN/pi3k.csv')
eval_pi3k